<a href="https://colab.research.google.com/github/bencode/tfadv/blob/main/Intro_to_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
example_tensor = torch.Tensor(
    [
     [[1, 2], [3, 4]], 
     [[5, 6], [7, 8]], 
     [[9, 0], [1, 2]]
    ]
)

In [3]:
example_tensor

tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]],

        [[9., 0.],
         [1., 2.]]])

In [4]:
example_tensor.device

device(type='cpu')

In [5]:
example_tensor.shape

torch.Size([3, 2, 2])

In [6]:
print("shape[0] =", example_tensor.shape[0])
print("size(1) =", example_tensor.size(1))

shape[0] = 3
size(1) = 2


In [7]:
print("Rank =", len(example_tensor.shape))
print("Number of elements =", example_tensor.numel())

Rank = 3
Number of elements = 12


In [8]:
example_tensor[1]

tensor([[5., 6.],
        [7., 8.]])

In [9]:
example_tensor[1, 1, 0]

tensor(7.)

In [10]:
example_scalar = example_tensor[1, 1, 0]
example_scalar.item()

7.0

In [11]:
example_tensor[:, 0, 0]

tensor([1., 5., 9.])

In [12]:
torch.ones_like(example_tensor)

tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]])

In [13]:
torch.zeros_like(example_tensor)

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [14]:
torch.randn_like(example_tensor)

tensor([[[ 0.0359,  0.0229],
         [ 1.8568,  0.0744]],

        [[ 0.5055, -0.1694],
         [ 0.9345,  0.6959]],

        [[-1.1081, -1.6198],
         [-0.1074, -1.1662]]])

In [17]:
torch.randn(2, 2, device='cuda') # Alternatively, for a GPU tensor, you'd use device='cuda'

tensor([[ 0.8512,  0.9256],
        [-0.9231,  0.3310]], device='cuda:0')

In [18]:
(example_tensor - 5) * 2

tensor([[[ -8.,  -6.],
         [ -4.,  -2.]],

        [[  0.,   2.],
         [  4.,   6.]],

        [[  8., -10.],
         [ -8.,  -6.]]])

In [19]:
print("Mean:", example_tensor.mean())
print("Stdev:", example_tensor.std())

Mean: tensor(4.)
Stdev: tensor(2.9848)


In [20]:
example_tensor.mean(0)

# Equivalently, you could also write:
# example_tensor.mean(dim=0)
# example_tensor.mean(axis=0)
# torch.mean(example_tensor, 0)
# torch.mean(example_tensor, dim=0)
# torch.mean(example_tensor, axis=0)

tensor([[5.0000, 2.6667],
        [3.6667, 4.6667]])

In [21]:
import torch.nn as nn

In [22]:
linear = nn.Linear(10, 2)
example_input = torch.randn(3, 10)
example_output = linear(example_input)
example_output

tensor([[-0.6607,  0.1885],
        [-0.8685, -0.5735],
        [-0.3218,  0.2426]], grad_fn=<AddmmBackward>)

In [23]:
relu = nn.ReLU()
relu_output = relu(example_output)
relu_output

tensor([[0.0000, 0.1885],
        [0.0000, 0.0000],
        [0.0000, 0.2426]], grad_fn=<ReluBackward0>)

In [24]:
batchnorm = nn.BatchNorm1d(2)
batchnorm_output = batchnorm(relu_output)
batchnorm_output

tensor([[ 0.0000,  0.4305],
        [ 0.0000, -1.3813],
        [ 0.0000,  0.9508]], grad_fn=<NativeBatchNormBackward>)

In [25]:
mlp_layer = nn.Sequential(
    nn.Linear(5, 2),
    nn.BatchNorm1d(2),
    nn.ReLU()
)

test_example = torch.randn(5,5) + 1
print("input: ")
print(test_example)
print("output: ")
print(mlp_layer(test_example))

input: 
tensor([[-1.1317, -0.7315,  0.7349,  1.9057, -0.4153],
        [ 0.9327,  0.9265,  1.2765,  0.3048,  0.2489],
        [ 0.3363,  1.6208,  3.5646,  1.2208,  1.3009],
        [ 1.8885,  0.4781,  1.1518,  0.5870, -0.1638],
        [ 2.0466,  0.5252,  1.3892,  2.4371,  0.7403]])
output: 
tensor([[0.0000, 1.0805],
        [0.0000, 0.8802],
        [0.0000, 0.0000],
        [0.6691, 0.1439],
        [1.3271, 0.0000]], grad_fn=<ReluBackward0>)


In [26]:
import torch.optim as optim
adam_opt = optim.Adam(mlp_layer.parameters(), lr=1e-1)

In [27]:
train_example = torch.randn(100,5) + 1
adam_opt.zero_grad()

# We'll use a simple loss function of mean distance from 1
# torch.abs takes the absolute value of a tensor
cur_loss = torch.abs(1 - mlp_layer(train_example)).mean()

cur_loss.backward()
adam_opt.step()
print(cur_loss)

tensor(0.7705, grad_fn=<MeanBackward0>)


In [28]:
class ExampleModule(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(ExampleModule, self).__init__()
        self.linear = nn.Linear(input_dims, output_dims)
        self.exponent = nn.Parameter(torch.tensor(1.))

    def forward(self, x):
        x = self.linear(x)

        # This is the notation for element-wise exponentiation, 
        # which matches python in general
        x = x ** self.exponent 
        
        return x

In [29]:
example_model = ExampleModule(10, 2)
list(example_model.parameters())

[Parameter containing:
 tensor(1., requires_grad=True), Parameter containing:
 tensor([[ 0.1930,  0.0982,  0.2016,  0.1974,  0.0592,  0.1147, -0.0805, -0.1470,
           0.2363, -0.1402],
         [ 0.0957, -0.2338, -0.1695, -0.1359, -0.1167,  0.0745, -0.0330, -0.2755,
           0.2197, -0.2767]], requires_grad=True), Parameter containing:
 tensor([0.2678, 0.2512], requires_grad=True)]

In [30]:
list(example_model.named_parameters())

[('exponent', Parameter containing:
  tensor(1., requires_grad=True)), ('linear.weight', Parameter containing:
  tensor([[ 0.1930,  0.0982,  0.2016,  0.1974,  0.0592,  0.1147, -0.0805, -0.1470,
            0.2363, -0.1402],
          [ 0.0957, -0.2338, -0.1695, -0.1359, -0.1167,  0.0745, -0.0330, -0.2755,
            0.2197, -0.2767]], requires_grad=True)), ('linear.bias',
  Parameter containing:
  tensor([0.2678, 0.2512], requires_grad=True))]

In [31]:
input = torch.randn(2, 10)
example_model(input)

tensor([[ 0.8279,  1.0500],
        [ 0.4653, -0.3032]], grad_fn=<PowBackward1>)